In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.globals import set_debug

# Variables

In [530]:
load_dotenv(override=True)

True

In [531]:
file = os.getenv("FILE")
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [532]:
loader = PyPDFLoader(file)
files = loader.load()

# Recursive TextSplit

In [533]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=30,
    length_function=len,
    separators= ["\n\n", "\n", ".", " "]
)

In [534]:
chunks = recur_split.split_documents(files)

# Embbiding (OpenAI)

In [535]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [536]:
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch

In [537]:
mongo_url = os.getenv("MONGO_URL")
mongo_db = os.getenv("MONGO_DB")
mongo_collection = os.getenv("MONGO_COLLECTION")

In [538]:
client = MongoClient(mongo_url)
db = client[mongo_db]
collection = db[mongo_collection]

In [539]:
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection=collection,          
    index_name="vector_index"
)

In [540]:
retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={
        'k': 3
    }
)

In [541]:
collection = retriever.vectorstore.collection
total_docs = collection.count_documents({})

print(f"Total de documentos na collection: {total_docs}")

Total de documentos na collection: 11


# LLM (OpenAI)

In [572]:
llm = ChatOpenAI(model="gpt-4o", api_key=api_key, temperature=0)

# Prompt

In [577]:
prompt = ChatPromptTemplate.from_template(
"""
Você é um assistente que deve responder a Pergunta baseada no Contexto informado.
O contexto e a pergunta do utilizador são apresentados a seguir.
Contexto: {context}
Pergunta: {input}

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
**Ano Letivo**;
**Disciplina**;
**Conteúdo**;
**Objetivo da Aula**.

Traga somente o(s) Codigo(s) e a(s) Descricao(s) que estao no Contexto e nada alem disso, nao invente descricao nem codigo.

Exemplo: \n
Pergunta: "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Calcular juros>." \n
Resposta: \n
Código: EF09MA05
Descrição: Resolver e elaborar problemas que envolvam porcentagens, com a ideia de aplicação de percentuais sucessivos e a determinação das taxas percentuais, preferencialmente com o uso de tecnologias digitais, no contexto da educação financeira. \n\n

"""
)


# Chain

In [578]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

documents_chain = create_stuff_documents_chain(llm, prompt)
retriver_chain = create_retrieval_chain(retriever, documents_chain)

# Question

In [579]:
question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Ponto medio> e o **Objetivo da Aula** é <Uso para calcular medidas>."

# Answer

In [580]:
retriver_chain.invoke({"input": question})

{'input': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Ponto medio> e o **Objetivo da Aula** é <Uso para calcular medidas>.',
 'context': [],
 'answer': 'Sinto muito, mas não posso ajudar com isso.'}